In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pywhisper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 KB 34.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.4 MB/s eta 0:00:00
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110742 sha256=254c9a70d1c5f935195589ef90a641eb34754bd99a852ce2372c52c70489aa59
  Stored in directory: /root/.cache/pip/wheels/29/15/e4/4f790bec6acd51a00b67e8ee1394f0bc6e0135c315f8ff399a
Successfully built moviepy
  Attempting uninstall: moviepy
    Found existing installation: moviepy 0.2.3.5
    Uninsta

In [4]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.5 MB/s eta 0:00:00


In [5]:
import pywhisper
from pywhisper.utils import write_srt, format_timestamp
from pathlib import Path

model_Large = pywhisper.load_model("large")

100%|██████████████████████████████████████| 2.87G/2.87G [00:14<00:00, 219MiB/s]


In [6]:
import csv
import os
from jiwer import wer

In [7]:
import re


In [8]:
working_directory = '/content/drive/MyDrive/Apziva/fake_audio_detection'

In [9]:
fake_voices_dir = os.path.join(working_directory, 'fake_speeches')

In [10]:
fake_list = os.listdir(fake_voices_dir)

In [11]:
fake_list[2]

'fake_1905.wav'

In [12]:
len(fake_list)

1968

In [13]:
not_found_files = []
for i in range(2001):
  if 'fake_{}.wav'.format(i) not in fake_list:
    #print(i)
    not_found_files.append(i)

In [14]:
len(not_found_files)

33

In [15]:
test_result_dict = model_Large.transcribe(os.path.join(fake_voices_dir,fake_list[2]))

In [16]:
test_result_dict['text']

' The publication of this report raised a storm in the city, and the Corporation was roused to make an immediate protest.'

In [17]:
clean_string = [s for s in test_result_dict['text'] if s.isalnum() or s.isspace()]
clean_string = "".join(clean_string)

In [18]:
clean_string

' The publication of this report raised a storm in the city and the Corporation was roused to make an immediate protest'

In [19]:
re.sub('\W+ ',' ', test_result_dict['text']).lower()

' the publication of this report raised a storm in the city and the corporation was roused to make an immediate protest.'

In [20]:
error_rate = wer(clean_string, clean_string)

In [21]:
print(error_rate)

0.0


In [22]:
error_list = []
path = fake_voices_dir

with open(os.path.join(working_directory, 'metadata.csv')) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='|')
    line_count = 0
    file_count = 0
    for row in csv_reader:
        if line_count in not_found_files:
          line_count+=1
          file_count+=1
          continue
        try:
            
            text_from_metadata=row[2]
            filename = fake_list[line_count]
            model_output = model_Large.transcribe(os.path.join(path,filename))

            model_output_text = model_output.get('text')
            #print(model_output_text)
      
            clean_string = [s for s in model_output_text if s.isalnum() or s.isspace()]
            clean_string = "".join(clean_string)
            clean_string = clean_string.lower()

            text_from_metadata = text_from_metadata.lower()
            text_from_metadata = [s for s in text_from_metadata if s.isalnum() or s.isspace()]
            text_from_metadata = "".join(text_from_metadata)
            error_rate = wer(text_from_metadata, clean_string)
            #tts.tts_to_file(text=text_from_metadata, file_path='fake_speeches/fake_{}.wav'.format(line_count))
            error_list.append(error_rate)
        except Exception as e:
            print('problem processing line: {}'.format(line_count))
            print(e)
        line_count+=1
        file_count+=1
        if file_count == len(fake_list):
            break
        
    print(f'Processed {line_count} lines.')

Processed 1968 lines.


In [23]:
import pickle

In [24]:
with open('error_list_pickled.pkl', 'wb') as f:
  pickle.dump(error_list, f)

In [25]:
def Average(lst):
    return sum(lst) / len(lst)

In [26]:
average = Average(error_list)

In [27]:
print(average)

1.2858963216978037
